In [1]:
# downloads all MPs and Senators' disclosures for the current parliament
# counts how many pages and how many words
# based on that, makes call on whether they are searchable.
# puts all that in a spreadsheet
# provides a searchable pdf

import datetime
today = datetime.datetime.now().strftime("%Y-%m-%d")
!mkdir $today
!mkdir $today/house
!mkdir $today/senate

mkdir: 2021-02-03: File exists
mkdir: 2021-02-03/house: File exists
mkdir: 2021-02-03/senate: File exists


In [43]:
# mps

import requests
from bs4 import BeautifulSoup
page = requests.get("https://www.aph.gov.au/Senators_and_Members/Members/Register")
soup = BeautifulSoup(page.content)
links = soup.find_all('a')
pdflinks = []
for l in links:
    try:
        if "pdf" in l.get("href"):
            if "46P" in l.get("href"):
                pdflinks.append(l.get("href"))
    except Exception as e:
        pass
        #print(e,l)
for p in pdflinks:
    name = p.split("/")[-1].split("?")[0]
    print(name)
    r = requests.get("http://aph.gov.au"+p, stream=True)
    with open(today+'/house/'+name, 'wb') as fd:
        for chunk in r.iter_content(2000):
            fd.write(chunk)

Albanese_46P.pdf
Alexander_46P.pdf
Allen_46P.pdf
Aly_46P.pdf
AndrewsKaren_46P.pdf
AndrewsKevin_46P.pdf
Archer_46P.pdf
Bandt_46P.pdf
Bell_46P.pdf
Bird_46P.pdf
Bowen_46P.pdf
Broadbent_46P.pdf
Buchholz_46P.pdf
Burke_46P.pdf
Burney_46P.pdf
Burns_46P.pdf
Butler_46P.pdf
ButlerM_46P.pdf
Byrne_46P.pdf
Chalmers_46P.pdf
Champion_46P.pdf
Chester_46P.pdf
ChestersL_46P.pdf
Christensen_46P.pdf
Clare_46P.pdf
Claydon_46P.pdf
Coker_46P.pdf
Coleman_46P.pdf
Collins_46P.pdf
Conaghan_46P.pdf
Connelly_46P.pdf
Conroy_46P.pdf
Coulton_46P.pdf
Dick_46P.pdf
Dreyfus_46P.pdf
Drum_46P.pdf
Dutton_46P.pdf
Elliot_46P.pdf
Entsch_46P.pdf
Evans__46P.pdf
Falinski_46P.pdf
Fitzgibbon_46P.pdf
Fletcher_46P.pdf
Flint_46P.pdf
Freelander_46P.pdf
Frydenberg_46P.pdf
Gee_46P.pdf
Georganas_46P.pdf
Giles_46P.pdf
Gillespie_46P.pdf
Goodenough_46P.pdf
Gorman_46P.pdf
Gosling_46P.pdf
Haines_46P.pdf
Hamilton_46P.pdf
Hammond_46P.pdf
Hastie_46P.pdf
Hawke_46P.pdf
Hayes_46P.pdf
Hill_46P.pdf
Hogan_46P.pdf
Howarth_46P.pdf
Hunt_46P.pdf
Husic_46P.

In [44]:
# senators

import requests
from bs4 import BeautifulSoup
page = requests.get("https://www.aph.gov.au/Parliamentary_Business/Committees/Senate/Senators_Interests/Register46thparl")
soup = BeautifulSoup(page.content)
links = soup.find_all('a')
pdflinks = []
for l in links:
    try:
        if "pdf" in l.get("href"):
            if "46th_Parl" in l.get("href"):
                pdflinks.append(l.get("href"))
    except Exception as e:
        pass
        #print(e,l)
for p in pdflinks:
    name = p.split("/")[-1].split("?")[0]
    print(name)
    r = requests.get("http://aph.gov.au"+p, stream=True)
    with open(today+'/senate/'+name, 'wb') as fd:
        for chunk in r.iter_content(2000):
            fd.write(chunk)

AbetzE_Astat_190726.pdf
AnticA_Astat_190719pdf.PDF
AskewW_Astat_190712.pdf
AyresT_Astat_190725.pdf
BilykC_Astat_190710.pdf
BirminghamS_Astat_190729.pdf
BraggA_Astat_190729.pdf
BrockmanS_Astat_190708.pdf
BrownC_Astat_190719.pdf
CanavanM_Astat_190729.pdf
CarrK_Astat_190704.pdf
CashM_Astat_190717.pdf
ChandlerC_Astat_190729.pdf
ChisholmA_Astat_190717.pdf
CicconeR_Astat_190719.pdf
ColbeckR_Astat_190730.pdf
DaveyP_Astat_1901719.pdf
DodsonP_Astat_190729.pdf
DuniamJ_Astat_190725.pdf
FarrellD_Astat_190729.pdf
FaruqiM_Astat_190730.pdf
FawcettD_Astat_190702.pdf
GallacherA_Astat_190722.pdf
GallagherK_Astat_190715.pdf
GriffS_Astat_190711.pdf
HansonP_Astat_190724.pdf
HansonYoungS_Astat_190723pdf.PDF
HughesH_Astat_190725.pdf
HumeJ_Astat_190730.pdf
LambieJ_Astat_190730.pdf
LinesS_Astat_190722.pdf
McAllisterJ_Astat_190730.pdf
McCarthyM_Astat_190729.pdf
McDonaldS_Astat_190730.pdf
McGrathJ_Astat_190730.pdf
McKenzieB_Astat_190726pdf.pdf
McKimN_Astat_190726.pdf
McLauchlanA_Astat_200303.pdf
McMahonS_Astat_1

In [45]:
import os
from PyPDF2 import PdfFileReader
import pandas as pd
import PyPDF2 
import textract, re
df = pd.DataFrame(columns=['file', 'chamber', 'pages', 'words'])
for root, dirs, files in os.walk(os.getcwd()):
    for f in files:
        if f.lower().endswith(".pdf"):
            pdf=PdfFileReader(open(os.path.join(root, f),'rb'))
            if "senate" in root:
                chamber = "senate"
            else:
                chamber = "house"
            text = textract.process(os.path.join(root, f),method='pdfminer')
            words = len(str(text).split(" "))
            df2 = pd.DataFrame([[f, chamber,pdf.getNumPages(),words]], columns=['file', 'chamber','pages','words'])
            df = df.append(df2, ignore_index=True)
df.to_csv(today+"/table.csv")

In [49]:
from PyPDF2 import PdfFileMerger

for chamber in ["house","senate"]:
    pdfs = os.listdir(today+"/"+chamber)

    merger = PdfFileMerger()

    for pdf in pdfs:
        #filename = today+"/"+chamber+"/"+pdf
        #print(filename)
        merger.append(filename,import_bookmarks=False )

    merger.write(today+"/"+chamber+".pdf")
    merger.close()

